<a href="https://colab.research.google.com/github/NikitaSUAI/EmotionRecognition/blob/main/FBank_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT

In [1]:
!pip install pytorch-lightning > /dev/null
!pip install torchmetrics > /dev/null
!pip install comet-ml > /dev/null
!pip install torch-ema > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [2]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# !cp -r '/content/drive/MyDrive/OMG_EMO' 'OMG_EMO'

BASE_PATH = Path("/content/drive/MyDrive/OMG_EMO")

TRAIN_PATH = BASE_PATH / "train_set.csv"
VAL_PATH = BASE_PATH / "val_set.csv"
TEST_PATH = BASE_PATH / "test_set.csv"

Mounted at /content/drive


In [3]:
import pandas as pd

train_df = pd.read_csv(TRAIN_PATH).dropna()
val_df = pd.read_csv(VAL_PATH).dropna()
test_df = pd.read_csv(TEST_PATH).dropna()

# Dataset

In [4]:
import torch
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [5]:
class FbanksDS(Dataset):
  def __init__(self, paths, dfs, atype=0, frame_len=None, aug=False):
    train_fbanks = dict()
    for path in paths:
      with open(path, "rb") as f:
        train_fbanks.update(pickle.load(f))
    df = pd.concat(dfs)
    x, clf, val, aro, reg = [], [], [], [], []
    row_path = []
    for idx, row in df.iterrows():
      x.append(train_fbanks[row.path])
      clf.append(row.EmotionMaxVote)
      reg.append(np.array((row.valence, row.arousal)))
      val.append(row.valence)
      aro.append(row.arousal)
      row_path.append(row.path)
    # self.x = x
    # self.label = [clf, val, aro]
    self.atype = atype
    self.frame_len = frame_len
    self.aug = aug
    tmp = [x_.shape[1] for x_ in x]
    x = [(x_, clf, val, aro, row_path) for _, x_, clf, val, aro, row_path in sorted(zip(tmp, x, clf, val, aro, row_path),  
                                              key=lambda pair: pair[0])][::-1]
    x, clf, val, aro, row_path  = list(map(list, zip(*x)))
    self.x = x
    self.label = [clf, val, aro]
    self.row_path = row_path
  
  def __len__(self):
    return len(self.label[0])
  
  def __getitem__(self, idx):
    # label = self.label[self.atype]
    labels = [label[idx] for label in self.label]
    feats = self.x[idx]
    # if self.frame_len is not None:
    #   feats = feats[:, :self.frame_len]
    #   if feats.shape[1] > self.frame_len:
    #     feats = feats[:, :self.frame_len]
    #   else:
    #     feats = [feats for _ in range(int(self.frame_len / feats.shape[1]) + 1)]
    #     feats = np.concatenate(feats, axis=1)[:, :self.frame_len]

    if np.random.uniform() < 0.05 and self.aug:
      feats = self.masking(feats)
    return feats[None, :, :], labels[0], labels[1], labels[2]

  def masking(self, feats):
    if np.random.uniform() > 0.5:
      x_start = np.random.randint(0, feats.shape[0]//2)
      x_end = np.random.randint(x_start, feats.shape[0]//2) * np.random.randint(1, 2)
      x_start = x_start * np.random.randint(1, 2)
      feats[x_start:x_end] = 0
    else:
      y_start = np.random.randint(0, feats.shape[1]//2)
      y_end = np.random.randint(y_start, feats.shape[1]//2) * np.random.randint(1, 2)
      y_start = y_start * np.random.randint(1, 2)
      feats[:, y_start:y_end] = 0
    return feats


In [6]:
def collate_fn(batch):
  
  feats, x, y, z = list(map(list, zip(*batch)))
  
  x, y, z = (
      torch.Tensor(x).long(),
      torch.Tensor(y).float(),
      torch.Tensor(z).float()
  )

  feats_len = [f.shape[2] for f in feats]
  max_len = min(feats_len)
  out_data = torch.zeros((len(feats), 1,  128, max_len))
  for idx, feat in enumerate(feats):
    f_len = feat.shape[1]
    out_data[idx, 0, :, :] = torch.from_numpy(feat[0, :, :max_len])
  return out_data.float(), x, y, z

# NN Model

In [7]:
def init_weight(distribution):
  def weights_init(m):
    if isinstance(m, nn.Linear):
      distribution(m.weight)
      m.bias.data.fill_(0.01)
    if isinstance(m, nn.Conv2d):
      distribution(m.weight)
    if type(m) in [nn.GRU, nn.LSTM, nn.RNN]:
      for name, param in m.named_parameters():
          if 'weight_ih' in name:
              torch.nn.init.xavier_uniform_(param.data)
          elif 'weight_hh' in name:
              torch.nn.init.orthogonal_(param.data)
          elif 'bias' in name:
              param.data.fill_(0.01)
    return m
  return weights_init

In [8]:
from torch import nn
from torchvision.models import resnet34, ResNet34_Weights, mobilenet_v3_small, efficientnet_b0, MobileNet_V3_Small_Weights

class CNN_model(nn.Module):
  def __init__(self, init_fn) -> None:
    super(CNN_model, self).__init__()
    self.frame_lvl = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)

    fc = init_fn(nn.Linear(576, 128))
    self.classifier = init_fn(nn.Linear(128, 7))
    self.val = init_fn(nn.Linear(128, 1))
    self.aro = init_fn(nn.Linear(128, 1))

    conv_1 = init_fn(nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, 
                       padding=1, stride=2, bias=False))
    # mv3
    self.frame_lvl.classifier[0] = fc
    self.frame_lvl.classifier = self.frame_lvl.classifier[:-1]
    self.frame_lvl.features[0][0] = conv_1

    # r32
    # self.frame_lvl.fc = fc
    # self.frame_lvl.conv1 = conv_1


  def forward(self, x):
    x = self.frame_lvl(x)
    clf_res = self.classifier(x)
    val = self.val(x)
    aro = self.aro(x)
    return clf_res, val, aro
  

# Metrics

In [9]:
from sklearn.metrics import f1_score
from scipy.stats import pearsonr

def ccc(y_true, y_pred, **kwargs):
    true_mean = np.mean(y_true)
    true_variance = np.var(y_true)
    pred_mean = np.mean(y_pred)
    pred_variance = np.var(y_pred)

    rho,_ = pearsonr(y_pred,y_true)

    std_predictions = np.std(y_pred)

    std_gt = np.std(y_true)


    ccc = 2 * rho * std_gt * std_predictions / (
        std_predictions ** 2 + std_gt ** 2 +
        (pred_mean - true_mean) ** 2)

    return float(ccc)

In [10]:
from torchmetrics import Metric
import numpy as np

class CCC_computer(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("y_hat", default=torch.Tensor([]), dist_reduce_fx="cat")
        self.add_state("y_true", default=torch.Tensor([]), dist_reduce_fx="cat")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        self.y_hat = torch.cat((self.y_hat.double(), preds[0].double()))
        self.y_true = torch.cat((self.y_true.double(), target[0].double()))

    def compute(self):
        res = ccc(self.y_true.detach().cpu().numpy(), self.y_hat.detach().cpu().numpy())
        return res

# Train Loop

In [13]:
from torch import optim, Tensor
import pytorch_lightning as pl
import torch.nn.functional as F
from torchmetrics import F1Score, MeanAbsoluteError
from torch_ema import ExponentialMovingAverage
import math


class OMG_model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = CNN_model(init_weight(nn.init.xavier_uniform))
        self.ema = ExponentialMovingAverage(self.model.parameters(), decay=0.95)

        self.metric_clf_val_micro = F1Score(num_classes = 7, average="micro")
        self.metric_clf_train_micro = F1Score(num_classes = 7, average="micro")

        self.metric_clf_val_macro = F1Score(num_classes = 7, average="macro")
        self.metric_clf_train_macro = F1Score(num_classes = 7, average="macro")

        self.metric_aro_val = CCC_computer()
        self.metric_aro_train = CCC_computer()

        self.metric_val_val = CCC_computer()
        self.metric_val_train = CCC_computer()

    def training_epoch_end(self, training_step_outputs):
        training_step_outputs = [x["loss"] for x in training_step_outputs]
        train_loss = torch.tensor(training_step_outputs).mean()

        train_clf_micro = self.metric_clf_train_micro.compute()
        self.metric_clf_train_micro.reset()

        train_clf_macro = self.metric_clf_train_macro.compute()
        self.metric_clf_train_macro.reset()

        train_aro = self.metric_aro_train.compute()
        self.metric_aro_train.reset()

        train_val = self.metric_val_train.compute()
        self.metric_val_train.reset()
        
        self.log("train_loss", torch.mean(train_loss))
        self.log("train_F1_micro", train_clf_micro)
        self.log("train_F1_macro", train_clf_micro)
        self.log("train_aro", train_aro)
        self.log("train_val", train_val)

    def training_step(self, batch, batch_idx):
        # x, y, z, h = batch
        x, y, z, h = list(map(lambda x: x.to(self.device), batch))
        y_hat, z_hat, h_hat = self.model(x)

        self.metric_clf_train_micro.update(y_hat, y)
        self.metric_clf_train_macro.update(y_hat, y)
        self.metric_aro_train.update(z_hat, z[:, None])
        self.metric_val_train.update(h_hat, h[:, None])


        # loss = F.cross_entropy(y_hat, y)
        # loss = F.mse_loss(z_hat, z[:, None].float())#, log_input=False)
        loss = F.mse_loss(h_hat, h[:, None].float())#, log_input=False)

        return loss

    def configure_optimizers(self):
        self.ema.to(self.device)
        self.model.to(self.device)
        optimizer = optim.AdamW(self.model.parameters())
        sheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99)
        return [optimizer], [sheduler]

    def validation_step(self, batch, batch_idx):
        # x, y, z, h = batch
        x, y, z, h = list(map(lambda x: x.to(self.device), batch))
        y_hat, z_hat, h_hat = self.model(x)
        
        # loss = F.cross_entropy(y_hat, y)
        # loss = F.mse_loss(z_hat, z[:, None].float())#, log_input=False)
        loss = F.mse_loss(h_hat, h[:, None].float())#, log_input=False)
        
        self.metric_clf_val_micro.update(y_hat, y)
        self.metric_clf_val_macro.update(y_hat, y)
        self.metric_aro_val.update(z_hat, z[:, None])
        self.metric_val_val.update(h_hat, h[:, None])
        return loss

    def validation_epoch_end(self, val_step_outputs):
        loss = torch.tensor(val_step_outputs).mean()

        val_clf_macro = self.metric_clf_val_macro.compute()
        self.metric_clf_val_macro.reset()

        val_clf_micro = self.metric_clf_val_micro.compute()
        self.metric_clf_val_micro.reset()

        val_aro = self.metric_aro_val.compute()
        self.metric_aro_val.reset()
        
        val_val = self.metric_val_val.compute()
        self.metric_val_val.reset()

        self.log("val_loss",  loss)
        self.log("val_F1_macro",    val_clf_macro)
        self.log("val_F1_micro",    val_clf_micro)
        self.log("val_aro",   val_aro)
        self.log("val_val",   val_val)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y, z, h = batch
        return self.model(x.float())

    def optimizer_step(self, *args, **kwargs):
      super().optimizer_step(*args, **kwargs)
      self.ema.update(self.model.parameters())

# Learning

In [20]:
train_dataset = FbanksDS(["/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_train.pkl",
                          "/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_test.pkl"],
                         [train_df, test_df], frame_len=1000, aug=True)

test_dataset = FbanksDS(["/content/drive/MyDrive/OMG_EMO/feats/fbanks_full_val.pkl"], [val_df], frame_len=None, aug=False)

In [21]:
idx = 0

In [22]:
import comet_ml
import random
import numpy as np
import torch
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import LearningRateMonitor

EXPIREMENT_NAME = f"FB_NoVAD_mv3_multytask_CE+MSE_{idx}"

idx += 1

comet_logger = CometLogger(
    api_key="*****",
    workspace="nikittossii",  # Optional
    save_dir=".",  # Optional
    project_name="omgemmotion",  # Optional
    experiment_name=EXPIREMENT_NAME,  # Optional
)

lr_monitor = LearningRateMonitor(logging_interval='step')

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

model = OMG_model()

train_dataloader = DataLoader(train_dataset, batch_size=128, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn)

trainer = pl.Trainer(limit_train_batches=128, max_epochs=13, accelerator="gpu",
                     logger=comet_logger, callbacks=[lr_monitor],
                     default_root_dir=f"/content/drive/MyDrive/model_results/{EXPIREMENT_NAME}/")


INFO:pytorch_lightning.loggers.comet:CometLogger will be initialized in online mode
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  import sys
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                   | Type         | Params
--------------------------------------------------------
0 | model                  | CNN_model    | 1.0 M 
1 | metric_clf_val_micro   | F1Score      | 0     
2 | metric_clf_train_micro | F1Score      | 0     
3 | metric_clf_val_macro   | F1Score      | 0     
4 | metric_clf_train_macro | F1Score      | 0     
5 | metric_aro_val         | CCC_computer | 0     
6 | metric_aro_train       | CCC_computer | 0     
7 | metric_val_val         | CCC_computer | 0     
8 | metric_val_train       | CCC_computer | 0     
--------------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.007     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn, torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml https://www.comet.com/nikittossii/omgemmotion/17bb566bd9ed47f9864f557c3e20d09d



...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=13` reached.
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nikittossii/omgemmotion/17bb566bd9ed47f9864f557c3e20d09d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr-AdamW [6]        : (0.0008953382542587163, 0.00099)
COMET INFO:     train_F1_macro [13] : (0.20602527260780334, 0.28474247455596924)
COMET INFO:     train_F1_micro [13] : (0.20602527260780334, 0.28474247455596924)
COMET INFO:     train_aro [13]      : (0.013248860836029053, 0.1690351963043213)
COMET INFO:     train_loss [13]     : (0.038013968616724014, 0.17236602306365967)
COMET INFO:     train_val [13]      : (-0.006649785675108433, 0.3936276435852051)
COMET INFO:     val_F1_macro [13]   : (0.03380648046731949, 0.10035336017608643)
COMET I

# Save results

In [26]:
EXPIREMENT_NAME = f"FB_NoVAD_mv3_multytask_CE+MSE"

In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn,)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn,)

prediction_train = trainer.predict(model, train_dataloader)
prediction_test = trainer.predict(model, test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 25it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn, torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Experiment is live on comet.ml https://www.comet.com/nikittossii/omgemmotion/17bb566bd9ed47f9864f557c3e20d09d

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nikittossii/omgemmotion/17bb566bd9ed47f9864f557c3e20d09d
COMET INFO:   Others:
COMET INFO:     Name : FB_

Predicting: 25it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn, torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Experiment is live on comet.ml https://www.comet.com/nikittossii/omgemmotion/17bb566bd9ed47f9864f557c3e20d09d

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nikittossii/omgemmotion/17bb566bd9ed47f9864f557c3e20d09d
COMET INFO:   Others:
COMET INFO:     Name : FB_

In [27]:
with open(f"/content/drive/MyDrive/OMG_EMO/unswers/{EXPIREMENT_NAME}_train.pkl", "wb") as f:
  pickle.dump([prediction_train, train_dataset.row_path], f)

with open(f"/content/drive/MyDrive/OMG_EMO/unswers/{EXPIREMENT_NAME}_test.pkl", "wb") as f:
  pickle.dump([prediction_test, test_dataset.row_path], f)